In [1]:
#import modules
import pandas as pd # for dataframes
import matplotlib.pyplot as plt # for plotting graphs
import seaborn as sns # for plotting graphs
import datetime as dt
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv("tran2_new.csv")

In [3]:
data.head()

,Unnamed: 0,date,customer_id,item_id,quantity,other_discount,coupon_discount,day,dow,month,coupon_used,year,selling_price
0,0,02-01-2012,1501,26830,1,-10.69,0.00,2,0,1,0,2012,14.892721
1,1,02-01-2012,464,20697,1,-21.37,-35.62,2,0,1,1,2012,36.828288
2,2,02-01-2012,464,20717,2,-13.89,0.00,2,0,1,0,2012,9.120373
3,3,02-01-2012,464,21008,1,-17.81,0.00,2,0,1,0,2012,17.200363
4,4,02-01-2012,464,22243,2,-11.40,0.00,2,0,1,0,2012,22.974331


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 13 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   Unnamed: 0       1048575 non-null  int64  
 1   date             1048575 non-null  object 
 2   customer_id      1048575 non-null  int64  
 3   item_id          1048575 non-null  int64  
 4   quantity         1048575 non-null  int64  
 5   other_discount   1048575 non-null  float64
 6   coupon_discount  1048575 non-null  float64
 7   day              1048575 non-null  int64  
 8   dow              1048575 non-null  int64  
 9   month            1048575 non-null  int64  
 10  coupon_used      1048575 non-null  int64  
 11  year             1048575 non-null  int64  
 12  selling_price    1048575 non-null  float64
dtypes: float64(3), int64(9), object(1)
memory usage: 104.0+ MB


In [5]:
data = data[(data['quantity']>0)]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1048575 entries, 0 to 1048574
Data columns (total 13 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   Unnamed: 0       1048575 non-null  int64  
 1   date             1048575 non-null  object 
 2   customer_id      1048575 non-null  int64  
 3   item_id          1048575 non-null  int64  
 4   quantity         1048575 non-null  int64  
 5   other_discount   1048575 non-null  float64
 6   coupon_discount  1048575 non-null  float64
 7   day              1048575 non-null  int64  
 8   dow              1048575 non-null  int64  
 9   month            1048575 non-null  int64  
 10  coupon_used      1048575 non-null  int64  
 11  year             1048575 non-null  int64  
 12  selling_price    1048575 non-null  float64
dtypes: float64(3), int64(9), object(1)
memory usage: 112.0+ MB


In [6]:
data.rename(columns={'Unnamed: 0':'Invoice No'},inplace= True)

In [7]:
data['TotalPurchase'] = data['quantity'] *data['selling_price']

In [16]:
data['customer_id'].nunique()

1582

In [9]:
data['date']= pd.to_datetime(data['date'])

In [10]:
data_group=data.groupby('customer_id').agg({'date': lambda date: (date.max() - date.min()).days,
                                            'Invoice No':'count',
                                        'quantity': lambda quant: quant.sum(),
                                        'TotalPurchase': lambda price: price.sum()})

In [11]:
data_group.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1582 entries, 1 to 1582
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           1582 non-null   int64  
 1   Invoice No     1582 non-null   int64  
 2   quantity       1582 non-null   int64  
 3   TotalPurchase  1582 non-null   float64
dtypes: float64(1), int64(3)
memory usage: 61.8 KB


In [190]:
# Change the name of columns
data_group.columns=['num_days','num_transactions','num_units','spent_money']
data_group.head()

,num_days,num_transactions,num_units,spent_money
customer_id,,,,
1,693,849,1010,30584.436378
2,576,292,336,11638.260841
3,667,624,1107,26139.025734
4,413,210,269,11527.108317
5,695,650,876,28095.099502


In [191]:
data_group['avg_order_value']=data_group['spent_money']/data_group['num_transactions']

In [192]:
data_group.head()

,num_days,num_transactions,num_units,spent_money,avg_order_value
customer_id,,,,,
1,693,849,1010,30584.436378,36.024071
2,576,292,336,11638.260841,39.857058
3,667,624,1107,26139.025734,41.889464
4,413,210,269,11527.108317,54.890992
5,695,650,876,28095.099502,43.223230


In [193]:
# Repeat Rate
repeat_rate=data_group[data_group.num_transactions > 1].shape[0]/data_group.shape[0]

In [194]:
repeat_rate

1.0

In [195]:
#Churn Rate
churn_rate=1-repeat_rate

In [196]:
purchase_frequency=sum(data_group['num_transactions'])/data_group.shape[0]

In [211]:
# Profit Margin
#Assuming 10% profit
data_group['profit_margin']=data_group['spent_money']*0.05

In [222]:
# Customer Value
#Considering churn rate 1,5,10
churn_rate=1
data_group['CLV1']=(data_group['avg_order_value']*purchase_frequency)/churn_rate

In [213]:
#Customer Lifetime Value
data_group['cust_lifetime_value1']=data_group['CLV1']*data_group['profit_margin']

In [214]:
data_group.head()

,num_days,num_transactions,num_units,spent_money,avg_order_value,profit_margin,CLV1,cust_lifetime_value1,CLV5,cust_lifetime_value5,CLV10,cust_lifetime_value10
customer_id,,,,,,,,,,,,
1,693,849,1010,30584.436378,36.024071,9175.330913,23877.332725,2.190824e+08,4775.466545,1.460550e+07,2387.733272,7.302748e+06
2,576,292,336,11638.260841,39.857058,3491.478252,26417.897758,9.223752e+07,5283.579552,6.149168e+06,2641.789776,3.074584e+06
3,667,624,1107,26139.025734,41.889464,7841.707720,27765.009511,2.177251e+08,5553.001902,1.451501e+07,2776.500951,7.257503e+06
4,413,210,269,11527.108317,54.890992,3458.132495,36382.630796,1.258160e+08,7276.526159,8.387731e+06,3638.263080,4.193865e+06
5,695,650,876,28095.099502,43.223230,8428.529851,28649.050822,2.414694e+08,5729.810164,1.609796e+07,2864.905082,8.048979e+06


In [215]:
# Customer Value
#Considering churn rate 1,5,10
churn_rate=5
data_group['CLV5']=(data_group['avg_order_value']*purchase_frequency)/churn_rate
#Customer Lifetime Value
data_group['cust_lifetime_value5']=data_group['CLV5']*data_group['profit_margin']
data_group.head()

,num_days,num_transactions,num_units,spent_money,avg_order_value,profit_margin,CLV1,cust_lifetime_value1,CLV5,cust_lifetime_value5,CLV10,cust_lifetime_value10
customer_id,,,,,,,,,,,,
1,693,849,1010,30584.436378,36.024071,9175.330913,23877.332725,2.190824e+08,4775.466545,4.381649e+07,2387.733272,7.302748e+06
2,576,292,336,11638.260841,39.857058,3491.478252,26417.897758,9.223752e+07,5283.579552,1.844750e+07,2641.789776,3.074584e+06
3,667,624,1107,26139.025734,41.889464,7841.707720,27765.009511,2.177251e+08,5553.001902,4.354502e+07,2776.500951,7.257503e+06
4,413,210,269,11527.108317,54.890992,3458.132495,36382.630796,1.258160e+08,7276.526159,2.516319e+07,3638.263080,4.193865e+06
5,695,650,876,28095.099502,43.223230,8428.529851,28649.050822,2.414694e+08,5729.810164,4.829388e+07,2864.905082,8.048979e+06


In [216]:
# Customer Value
#Considering churn rate 1,5,10
churn_rate=10
data_group['CLV10']=(data_group['avg_order_value']*purchase_frequency)/churn_rate
#Customer Lifetime Value
data_group['cust_lifetime_value10']=data_group['CLV10']*data_group['profit_margin']
data_group.head()

,num_days,num_transactions,num_units,spent_money,avg_order_value,profit_margin,CLV1,cust_lifetime_value1,CLV5,cust_lifetime_value5,CLV10,cust_lifetime_value10
customer_id,,,,,,,,,,,,
1,693,849,1010,30584.436378,36.024071,9175.330913,23877.332725,2.190824e+08,4775.466545,4.381649e+07,2387.733272,2.190824e+07
2,576,292,336,11638.260841,39.857058,3491.478252,26417.897758,9.223752e+07,5283.579552,1.844750e+07,2641.789776,9.223752e+06
3,667,624,1107,26139.025734,41.889464,7841.707720,27765.009511,2.177251e+08,5553.001902,4.354502e+07,2776.500951,2.177251e+07
4,413,210,269,11527.108317,54.890992,3458.132495,36382.630796,1.258160e+08,7276.526159,2.516319e+07,3638.263080,1.258160e+07
5,695,650,876,28095.099502,43.223230,8428.529851,28649.050822,2.414694e+08,5729.810164,4.829388e+07,2864.905082,2.414694e+07


In [217]:
data_group.to_csv("cltv1_5_10-30.csv",index= False)

In [286]:
data['month_yr'] = data['date'].apply(lambda x: x.strftime('%b-%Y'))

In [302]:
data.head()

,Invoice No,date,customer_id,item_id,quantity,other_discount,coupon_discount,day,dow,month,coupon_used,year,selling_price,TotalPurchase,month_yr
0,0,2012-02-01,1501,26830,1,-10.69,0.00,2,0,1,0,2012,14.892721,14.892721,Feb-2012
1,1,2012-02-01,464,20697,1,-21.37,-35.62,2,0,1,1,2012,36.828288,36.828288,Feb-2012
2,2,2012-02-01,464,20717,2,-13.89,0.00,2,0,1,0,2012,9.120373,18.240747,Feb-2012
3,3,2012-02-01,464,21008,1,-17.81,0.00,2,0,1,0,2012,17.200363,17.200363,Feb-2012
4,4,2012-02-01,464,22243,2,-11.40,0.00,2,0,1,0,2012,22.974331,45.948661,Feb-2012


In [303]:
sale=data.pivot_table(index=['customer_id'],columns=['month_yr'],values='TotalPurchase',aggfunc='sum',fill_value=0).reset_index()

In [304]:
sale.head()

month_yr,customer_id,Apr-2012,Apr-2013,Aug-2012,Aug-2013,Dec-2012,Dec-2013,Feb-2012,Feb-2013,Jan-2012,...,Mar-2012,Mar-2013,May-2012,May-2013,Nov-2012,Nov-2013,Oct-2012,Oct-2013,Sep-2012,Sep-2013
0,1,1981.992610,1053.908731,2171.653594,0.000000,3442.944189,1521.579050,1018.114345,2180.310495,1296.697349,...,2829.104816,863.891230,1839.291502,0.000000,2369.533934,0.000000,1461.194659,988.792377,1202.599339,0.000000
1,2,732.738057,0.000000,278.339924,93.394698,0.000000,0.000000,0.000000,415.725676,1553.918455,...,241.394957,0.000000,752.709535,0.000000,977.950995,0.000000,649.264277,0.000000,1874.625656,0.000000
2,3,516.506774,0.000000,3532.326441,149.390679,1416.639658,0.000000,0.000000,703.173008,1120.648862,...,0.000000,191.174564,4179.043236,0.000000,3481.512932,845.177287,679.976664,0.000000,2548.353536,0.000000
3,4,2201.921955,0.000000,466.292863,0.000000,248.551241,0.000000,0.000000,356.501588,434.083101,...,345.410644,0.000000,2041.557010,0.000000,472.288844,0.000000,1502.301160,0.000000,1012.004926,0.000000
4,5,1487.982109,218.422247,1894.927724,313.784274,3312.056784,499.769884,779.030272,885.013288,228.002852,...,387.664484,563.719453,3669.232515,747.750697,3370.797952,286.189797,1121.903157,0.000000,2769.206586,312.860569


In [305]:
sale['CLV1']=sale.iloc[:,2:].sum(axis=1)

In [306]:
sale.head()

month_yr,customer_id,Apr-2012,Apr-2013,Aug-2012,Aug-2013,Dec-2012,Dec-2013,Feb-2012,Feb-2013,Jan-2012,...,Mar-2013,May-2012,May-2013,Nov-2012,Nov-2013,Oct-2012,Oct-2013,Sep-2012,Sep-2013,CLV1
0,1,1981.992610,1053.908731,2171.653594,0.000000,3442.944189,1521.579050,1018.114345,2180.310495,1296.697349,...,863.891230,1839.291502,0.000000,2369.533934,0.000000,1461.194659,988.792377,1202.599339,0.000000,28602.443767
1,2,732.738057,0.000000,278.339924,93.394698,0.000000,0.000000,0.000000,415.725676,1553.918455,...,0.000000,752.709535,0.000000,977.950995,0.000000,649.264277,0.000000,1874.625656,0.000000,10905.522785
2,3,516.506774,0.000000,3532.326441,149.390679,1416.639658,0.000000,0.000000,703.173008,1120.648862,...,191.174564,4179.043236,0.000000,3481.512932,845.177287,679.976664,0.000000,2548.353536,0.000000,25622.518960
3,4,2201.921955,0.000000,466.292863,0.000000,248.551241,0.000000,0.000000,356.501588,434.083101,...,0.000000,2041.557010,0.000000,472.288844,0.000000,1502.301160,0.000000,1012.004926,0.000000,9325.186362
4,5,1487.982109,218.422247,1894.927724,313.784274,3312.056784,499.769884,779.030272,885.013288,228.002852,...,563.719453,3669.232515,747.750697,3370.797952,286.189797,1121.903157,0.000000,2769.206586,312.860569,26607.117393


In [307]:
X=sale[['Jul-2013','Aug-2013','Sep-2013','Oct-2013','Nov-2013']]
y=sale[['CLV1']]

In [308]:
#split training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=0)

In [309]:
# import model
from sklearn.linear_model import LinearRegression

# instantiate
linreg = LinearRegression()

# fit the model to the training data (learn the coefficients)
linreg.fit(X_train, y_train)

# make predictions on the testing set
y_pred1 = linreg.predict(X_test)

In [310]:
# print the intercept and coefficients
print(linreg.intercept_)
print(linreg.coef_)

[13602.73913299]
[[ 9.09395657  9.69209519 12.81717187 11.22915248 10.83641377]]


In [311]:
from sklearn import metrics

# compute the R Square for model
print("R-Square:",metrics.r2_score(y_test, y_pred1))

R-Square: 0.5241522759173942


In [316]:
y_pred5
y_pred5=pd.DataFrame(data=y_pred5)

In [315]:
y_pred10
y_pred10=pd.DataFrame(data=y_pred10)

In [299]:
y_pred1


array([[14576.38557872],
       [13851.71604756],
       [32314.73008184],
       [50931.79480295],
       [16573.15453563],
       [14164.48190079],
       [63180.63183542],
       [13602.73913299],
       [13602.73913299],
       [16904.95175119],
       [57153.14802601],
       [30872.79147176],
       [16346.29880835],
       [47495.88150446],
       [13602.73913299],
       [14722.81561177],
       [28740.31071034],
       [13602.73913299],
       [35766.70141937],
       [15561.82113296],
       [17799.97923503],
       [37554.25091947],
       [28033.86646582],
       [17116.70924263],
       [19007.27904472],
       [33386.18821168],
       [30247.15368537],
       [24735.01517084],
       [31584.89350137],
       [13602.73913299],
       [13602.73913299],
       [61622.93529292],
       [25276.89348735],
       [13602.73913299],
       [63245.98661407],
       [13602.73913299],
       [20452.53635909],
       [15133.62372602],
       [28453.45267249],
       [52274.39922863],


In [318]:
y_pred1=pd.DataFrame(data=y_pred1)

In [319]:
y_pred1

,0
0,14576.385579
1,13851.716048
2,32314.730082
3,50931.794803
4,16573.154536
...,...
391,32770.296411
392,17008.352485
393,17789.734934
394,15702.933923


In [320]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1582 entries, 0 to 1581
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Jul-2013  1582 non-null   float64
 1   Aug-2013  1582 non-null   float64
 2   Sep-2013  1582 non-null   float64
 3   Oct-2013  1582 non-null   float64
 4   Nov-2013  1582 non-null   float64
dtypes: float64(5)
memory usage: 61.9 KB


In [321]:
y_pred1

,0
0,14576.385579
1,13851.716048
2,32314.730082
3,50931.794803
4,16573.154536
...,...
391,32770.296411
392,17008.352485
393,17789.734934
394,15702.933923


In [322]:
y_pred1['y_pred5']=y_pred5

In [323]:
y_pred1['y_pred10']=y_pred10

In [324]:
y_pred1

,0,y_pred5,y_pred10
0,14576.385579,29152.771157,58305.542315
1,13851.716048,27703.432095,55406.864190
2,32314.730082,64629.460164,129258.920327
3,50931.794803,101863.589606,203727.179212
4,16573.154536,33146.309071,66292.618143
...,...,...,...
391,32770.296411,65540.592822,131081.185643
392,17008.352485,34016.704970,68033.409940
393,17789.734934,35579.469868,71158.939737
394,15702.933923,31405.867847,62811.735694


In [325]:
y_pred1.to_csv('Predicted_y_values.csv')